# The LSTM Model

Version Control:
- Last Trial
- 8 CA counties
- Averaged on 5 runs
- LSTM with 1 layer

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

In [ ]:
# import data
df = pd.read_csv("CACases.csv", index_col = False)
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# counties we are interested in
counties = ["Los Angeles", "San Diego", "San Francisco", "Santa Barbara",\
            "Fresno", "Sacramento", "Ventura", "Riverside"]

# Data Smoothing
- Remove day of week effect by replace $y_t$ with an average of 7 previous days(inclusive)
    - Each value is an average of data from all 7 different weekdays, thud day of week effect removed

In [ ]:
# create a time series of cases, each value is an average of 7 previous days(include the current date)
# for each county
df1 = df.copy()  # deep copy
df1 = df1.iloc[6:]
for county in counties:
    # holder
    temp = []
    # take average, starting from 7th day
    for i in range(6,len(df)):
        # remove day of week effect
        ave = np.mean(df[county].iloc[i-6:i+1])  # 6 previous days and the exact date, 7 days' average
        temp.append(ave)
    df1[county] = temp

In [ ]:
df1.set_index("date", inplace=True)

In [ ]:
# input: original dataset, county we want
# output: all data, sorted by date, of that county
def extract(dataset, county):
    # select data only from the input county
    temp = dataset.to_dict()[county]
    temp = pd.Series(temp)
    # reset index
    # temp.reset_index(drop=True, inplace=True)
    return temp

# Data Preprocessing
## Differencing
- Differencing of time series in discrete time
- transformation of series to a new time series wehre values are the difference between consecutive values of previous series
- help stablizing the mean of time series by removing trend

In [ ]:
# create a differenced series, output as Series
# interval: order of differencing, default = 1
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        # no need to difference
        if (interval == 0):
            value = dataset[i]
        else:
            value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)
# invert differenced value
# input: single original, single prediction, interval(default as 1)
def invert_difference(history, yhat, interval=1):
    if (interval == 0):
        return yhat
    else:
        return yhat + history

## Scaling
- Scale the data to [-1,1]

In [ ]:
# scale train and test data to [-1, 1]
# input: train(np array, train_size*1), test(np array, test_size*1)
def scale(train, test):
    # find scalers
    aveTrain = np.mean(train)
    maxTrain = max(train)
    minTrain = min(train)
    scaler = [aveTrain, maxTrain, minTrain]
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = (train - aveTrain)/(maxTrain-minTrain)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = (test - aveTrain)/(maxTrain-minTrain)
    return scaler, train_scaled, test_scaled
# inverse scaling for a forecasted value
# input: scaler, single prediction yhat
def invert_scale(scaler, yhat):
    inverted = yhat*(scaler[1]-scaler[2]) + scaler[0]
    return inverted[0][0]  # output: a number

## Reshaping

In [ ]:
# convert to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = [], []
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
# transform series into train and test sets for supervised learning
def prepare_data(series, diff, train_size, test_size, n_lag, n_seq):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), 1)
    # differencing
    diff_series = difference(raw_values, diff)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
    # split into train & test
    train_diff, test_diff = diff_values[:(train_size-diff)], diff_values[(train_size-diff):]
    # rescaling
    scaler, train_scaled, test_scaled = scale(train_diff, test_diff)
    # adjust data type for train_scaled, test_scaled
    train_scaled = [row[0][0] for row in train_scaled]
    test_scaled = [row[0][0] for row in test_scaled]
    # convert to supervised
    train = series_to_supervised(train_scaled, n_lag, n_seq).values
    test = series_to_supervised(test_scaled, n_lag, n_seq).values
    return scaler, train, test  # scaler: list of np array, train: np array, test: np array

# Long Short-Term Memory(LSTM)
- The Long Short-Term Memory network or LSTM is a recurrent neural network that can learn and forecast long sequences.
- Walk forward model validation: a prediction will be made for a time step, then the real value used for model to predict next time step
    - stimulates the reality
- RMSE will be used: punishes large errors and results in a score that is in the same units as the forecast data

In [ ]:
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = keras.Sequential()
    # single layer lstm
    model.add(layers.LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(layers.Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

# Prediction and Evaluation

In [ ]:
# make one forecast with an LSTM
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # convert to array
    return [x for x in forecast[0, :]][0]
# evaluate the model
def make_forecasts(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff):
    forecasts = []
    prev = raw_value[-(test_size+diff):(len(raw_value)-diff)]  # Y_(t-diff)
    for i in range(len(test)):
        X = test[i, 0:n_lag]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # invert scaling
        forecast = invert_scale(scaler, forecast)
        # invert differencing
        forecast = invert_difference(prev[i], forecast)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [ ]:
# evaluate forcasts
def evaluate_forecasts(actual, forecasts):
    rmse = np.sqrt(mean_squared_error(actual, forecasts))
    mape = mean_absolute_percentage_error(actual, forecasts)*100
    return rmse, mape

## A Single Trial as Demo

In [ ]:
# given historical observations (t, t-1, t-2, … t-n+1) forecast t+1(future 1 day)
n_lag = 7  # number of lag, use 7 past days
n_seq = 1  # predict 1 future days
diff = 1 # order of difference
train_size = 63  # train set size
test_size = 18 # test set size

n_batch = 1
nb_epoch = 100
n_neurons = 1

In [ ]:
county = "Los Angeles"

In [ ]:
# prepare data
s = extract(df1, county)
raw_value = s[-(train_size+test_size+n_lag):]
scaler, train, test = prepare_data(raw_value, diff, train_size, test_size, n_lag, n_seq)

In [ ]:
# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
# make forecasts
forecasts = make_forecasts(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
evaluate_forecasts(raw_value[-test_size:], forecasts)

In [ ]:
truth = raw_value[-test_size:].to_list()
plt.plot(truth, color = 'blue', label = "ground truth")
plt.plot(forecasts, color = 'orange', label = "prediction")
plt.legend()
plt.savefig("LSTM_demo.png", dpi=600)

# Evaluate LSTM Method on All Counties, LAST Trial

In [ ]:
# for each date t
    # use [t, t + train_size) as training set
    # use [t + train_size, t + train_size + test_size) as testing set
    # evaluate by calculating RMSE

# holder for metrics:
RMSE = []
MAPE = []
trainingTime = []
for county in counties:
    # cut a data trial from original dataset
    # prepare data
    s = extract(df1, county)
    raw_value = s[-(train_size+test_size+n_lag):]
    scaler, train, test = prepare_data(raw_value, diff, train_size, test_size, n_lag, n_seq)
    # for each dataset, fit model 5 time, take average    
    rmse, mape, training_time = 0, 0, 0
    for i in range(5):
        tic = time.perf_counter()
        model = fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
        toc = time.perf_counter()
        # make forecasts
        forecasts = make_forecasts(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
        temp = evaluate_forecasts(raw_value[-test_size:], forecasts)
        rmse += temp[0]
        mape += temp[1]
        training_time += (toc - tic)
    RMSE.append(rmse/5)
    MAPE.append(mape/5)
    trainingTime.append(training_time/5)

In [ ]:
LSTM_evaluate = {'county': counties, 'rmse': RMSE, 'mape': MAPE, "training time": trainingTime}
LSTM_evaluate = pd.DataFrame(LSTM_evaluate)
LSTM_evaluate

# Perform on All Counties, All Trials

In [ ]:
# given historical observations (t, t-1, t-2, … t-n+1) forecast t+1(future 1 day)
n_lag = 7  # number of lag, use 7 past days
n_seq = 1  # predict 1 future days
diff = 1 # order of difference
train_size = 63  # train set size
test_size = 18 # test set size

n_step = 7  # step size

n_batch = 1
nb_epoch = 100
n_neurons = 1

In [ ]:
# holder for rmse:
RMSE = []
MAPE = []

for county in counties:
    s = extract(df1, county)
    temp_RMSE = []
    temp_MAPE = []
    for j in range(0, len(df1)-(train_size+test_size+n_lag)+1, n_step):
        # prepare data
        raw_value = s[j:j+(train_size+test_size+n_lag)]
        
        # for double layer lstm
        scaler, train, test = prepare_data(raw_value, diff, train_size, test_size, n_lag, n_seq)
        model = fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
        forecasts = make_forecasts(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
        rmse_LSTM, mape_LSTM = evaluate_forecasts(raw_value[-test_size:], forecasts)
      
        temp_RMSE.append(rmse_LSTM)
        temp_MAPE.append(mape_LSTM)
    
    RMSE.append(temp_RMSE)
    MAPE.append(temp_MAPE)
    print(county)

In [ ]:
for i in range(len(counties)):
    county = counties[i]
    temp_RMSE = RMSE[i]
    temp_MAPE = MAPE[i]
    
    curr = {"rmse": temp_RMSE, "mape": temp_MAPE}
    curr = pd.DataFrame(curr)
    curr.to_csv(county+"_single_lstm.csv", index = False)

In [ ]:
curr

In [ ]:
ave_rmse = []
ave_mape = []
for i in range(len(counties)):
    county = counties[i]
    temp_RMSE = RMSE[i]
    temp_MAPE = MAPE[i]
    
    ave_rmse.append(np.mean(temp_RMSE))
    ave_mape.append(np.mean(temp_MAPE))

LSTM = {"county": counties, "rmse": ave_rmse, "mape": ave_mape}
LSTM = pd.DataFrame(LSTM)

In [ ]:
LSTM